# <b style="color:blue">MultiLayer Perceptron</b>
A multilayer perceptron is a class of feedforward artificial neural network. The term MLP is used ambiguously, sometimes loosely to any feedforward ANN, sometimes strictly to refer to networks composed of multiple layers of perceptrons.

<img src="https://www.researchgate.net/profile/Dominique_Guerillot3/publication/314106922/figure/fig11/AS:653106945724416@1532724238105/Example-of-multilayer-perceptron-with-two-hidden-layers.png" alt = "MLP">

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
housing_datasets = fetch_california_housing()

In [3]:
x_train_full, x_test, y_train_full, y_test = train_test_split(housing_datasets.data, housing_datasets.target)
x_train, x_validation, y_train, y_validation = train_test_split(x_train_full, y_train_full)

# Why we use StandardScaler()

StandardScaler removes the mean and scales each feature/variable to unit variance. This operation is performed feature-wise in an independent way. StandardScaler can be influenced by outliers (if they exist in the dataset) since it involves the estimation of the empirical mean and standard deviation of each feature.

In [4]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_validation = scaler.fit_transform(x_validation)
x_test = scaler.fit_transform(x_test)

Using the Sequential API to build, train, evaluate, and use a regression MLP to make
predictions is quite similar to what we did for classification. The main differences are
the fact that the output layer has a single neuron (since we only want to predict a single
value) and uses no activation function, and the loss function is the mean squared
error. Since the dataset is quite noisy, we just use a single hidden layer with fewer
neurons than before, to avoid overfitting:

# Create Sequential API model

In [5]:
model = tf.keras.Sequential([tf.keras.layers.Dense(30, activation = 'relu', input_shape = x_train.shape[1:]),
                            tf.keras.layers.Dense(1)])


# Compile The Model

In [6]:
model.compile(loss = tf.losses.mean_squared_error,
             optimizer = 'sgd')

# Fit the Model

In [7]:
hostory = model.fit(x_train, y_train, epochs=20,
                   validation_data=(x_validation, y_validation))

Epoch 1/20
363/363 [==============================] - 0s 1ms/step - loss: 1.2497 - val_loss: 0.7307
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7161 - val_loss: 0.6768
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5175 - val_loss: 0.6943
Epoch 4/20
363/363 [==============================] - 0s 997us/step - loss: 0.3963 - val_loss: 0.7604
Epoch 5/20
363/363 [==============================] - 0s 987us/step - loss: 0.3879 - val_loss: 0.8083
Epoch 6/20
363/363 [==============================] - 0s 991us/step - loss: 0.3810 - val_loss: 0.8629
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3757 - val_loss: 0.9550
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3705 - val_loss: 1.0322
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3669 - val_loss: 1.1524
Epoch 10/20
363/363 [==============================] - 0s 990us/step - loss: 0.3611 - val_loss

# Test the model

In [8]:
mean_squared_error_test = model.evaluate(x_test, y_test)
x_new = x_test[:3] # New instance
y_pred = model.predict(x_new)
y_pred

162/162 [==============================] - 0s 693us/step - loss: 0.3733


array([[2.888164  ],
       [2.2869391 ],
       [0.70185906]], dtype=float32)

As you can see, the Sequential API is quite easy to use. However, although Sequen
tial models are extremely common, it is sometimes useful to build neural networks
with more complex topologies, or with multiple inputs or outputs. For this purpose,
Keras offers the Functional API.

# Building Complex Models Using The Functional API

# Functional API

The <b style="color:blue"> functional API </b> can handle models with non-linear topology, shared layers, and even multiple inputs or outputs. The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.

<img src="https://i.stack.imgur.com/WCess.png" alt = "Functional API and Sequential API">

# Build a Neural Network to Handel the California Housing Datasets problem:

In [9]:
input_a = tf.keras.layers.Input(shape = [5], name = 'wide_input')
input_b = tf.keras.layers.Input(shape = [6], name = 'deep_input')
hidden1 = tf.keras.layers.Dense(30, activation = 'relu')(input_b)
hidden2 = tf.keras.layers.Dense(30, activation = 'relu')(hidden1)
concat = tf.keras.layers.concatenate([input_a, hidden2])
output = tf.keras.layers.Dense(1, name = 'output')(concat)

In [10]:
model = tf.keras.Model(inputs = [input_a, input_b], outputs = [output])

In [11]:
model.compile(loss = tf.losses.mean_squared_error,
             optimizer = tf.keras.optimizers.SGD(lr = 1e-3))

In [12]:
x_train_a, x_train_b = x_train[:, :5], x_train[:, 2:]
x_validation_a, x_validation_b = x_validation[:, :5], x_validation[:, 2:]
x_test_a, x_test_b = x_test[:, :5], x_test[:, 2:]
x_new_a, x_new_b = x_test_a[:3], x_test_b[:3]

In [13]:
history = model.fit((x_train_a, x_train_b), y_train, epochs=20, validation_data=((x_validation_a, x_validation_b), y_validation))
mean_squared_error_test = model.evaluate((x_test_a, x_test_b), y_test)
y_pred = model.predict((x_new_a, x_new_b))

Epoch 1/20
363/363 [==============================] - 0s 1ms/step - loss: 1.7420 - val_loss: 0.8712
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7707 - val_loss: 0.7092
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6689 - val_loss: 0.6366
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6182 - val_loss: 0.5900
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5836 - val_loss: 0.5556
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5574 - val_loss: 0.5309
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5372 - val_loss: 0.5148
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5219 - val_loss: 0.5037
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5102 - val_loss: 0.4936
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5008 - val_loss: 0.4872

Both models are pretty close 📚